In [11]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By #per cercare id html
from selenium.webdriver.common.keys import Keys #per usare tastiera
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

import csv
import os
from dotenv import load_dotenv

In [15]:
#Aggiunge valori al CSV
def addToCSV(listToAppend, appName):
    filename = 'student-apk.csv'
    rows = []
    
    with open(filename, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames  #Ottieni colonne
        
        for row in reader:
            #Aggiorna riga dove trova nome
            if row['pkg_name'] == appName:
                row['data_safety'] = listToAppend
            rows.append(row)

    #Scrivi di nuovo il file CSV con il contenuto aggiornato
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)


In [16]:
#get gli id per data safety 
def getDataSafety(driver, appName):
    try:
        #Aspetta fino a che non trova elemento
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CLASS_NAME, "aFEzEb"))
        )
        l = driver.find_elements(By.CLASS_NAME, "aFEzEb")

        #Estrai testo con elementi trovati
        listElements = [x.text for x in l]
        
        #Aggiunge
        addToCSV(listElements, appName)

    except TimeoutException:
        #Se non trova aggiunge NOT USED
        addToCSV("NOT USED", appName)

In [17]:

def lookForApp(appName):
    load_dotenv()
    
    chromeDriverPath = os.getenv('webdriver_chrome_driver')
    
    #Connecting to play store
    service = Service(executable_path=chromeDriverPath)
    #service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    

    driver.get("https://play.google.com/store/apps/datasafety?id=" + appName)
    getDataSafety(driver, appName)




In [14]:
#Legge i dati nomi da pkg_name
filename = 'student-apk.csv'




with open(filename, mode='r') as file:
    csv_reader = csv.DictReader(file) #Per mappare i nomi delle colonne
    for row in csv_reader:
        #Estrai pkg_name
        pkg_name = row['pkg_name']
        lookForApp(pkg_name)

